# Analysis 2e6 muons by CJ - ROI cut
In this notebook, we will see the last data from CJ. Only the entries within the ROI are considered. The ROI is defined as follows:
- `x`, `y` in `[-500, +500]`
- `z` in `[-1000, +1000]`

In [11]:
dir_in = os.path.join("..", "Data", "cutROI")
files = ["output2eROI_part1.csv", "output2eROI_part2.csv", "output2eROI_part3.csv", "output2eROI_part4.csv", "output2eROI_part5.csv"]
df = pd.read_csv(os.path.join(dir_in, files[0]))
for file in files[1:]:
    dfi = pd.read_csv(os.path.join(dir_in, file))
    df = pd.concat([df, dfi])

423347


In [18]:
print("[Info] Number of entries: {}".format(df.shape[0]))
print("[Info] Number of distinct events: {}".format(df.eventnumber.nunique()))

[Info] Number of entries: 9324212
[Info] Number of distinct events: 423347


In [16]:
df.shape



(9324212, 15)